# New Maximal Matching
ของเดิม ใช้ multicut ก่อน ทำให้เสียเวลา ตัวใหม่ทำ maximal โดยตรง จะได้เร็วขึ้น

## Import

In [ ]:
import re
from collections import defaultdict
from heapq import heappush, heappop  # for priority queue

!pip install -q marisa_trie
from marisa_trie import Trie

## Load Data
เอา wordlist มาสร้าง trie

In [ ]:
!wget -nv https://github.com/PyThaiNLP/pythainlp/raw/dev/pythainlp/corpus/thaiword.txt

2018-02-19 04:20:01 URL:https://raw.githubusercontent.com/PyThaiNLP/pythainlp/dev/pythainlp/corpus/thaiword.txt [1245519/1245519] -> "thaiword.txt" [1]


In [ ]:
wordlist = [li.strip() for li in open('thaiword.txt')]
trie = Trie(wordlist)

## One Cut
แทนที่ multicut

In [ ]:
# ช่วยตัดพวกภาษาอังกฤษ เป็นต้น
pat_eng = re.compile(r'''(?x)
[-a-zA-Z]+|   # english
\d[\d,\.]*|   # number
[ \t]+|       # space
\r?\n         # newline
''')

In [ ]:
text = 'สวัสดีครับ สบายดีไหมครับ'

In [ ]:
def onecut(text):
  words_at = defaultdict(list)  # main data structure

  def serialize(p, p2):    # helper function
    for w in words_at[p]:
      p_ = p + len(w)
      if p_== p2:
        yield [w]
      elif p_ < p2:
        for path in serialize(p_, p2):
          yield [w]+path

  q = [0]       # min-heap queue
  last_p = 0    # last position for yield
  while q[0] < len(text):
      p = heappop(q)

      for w in trie.prefixes(text[p:]):
          words_at[p].append(w)
          if p+len(w) not in q:
            heappush(q, p+len(w))

      if len(q)==1:
          for w in min(serialize(last_p, q[0]), key=len):
            yield w
          last_p = q[0]

      # กรณี len(q) == 0  คือ ไม่มีใน dict
      if len(q)==0:
          m = pat_eng.match(text[p:])
          if m: # อังกฤษ, เลข, ว่าง
              i = p + m.span()[1]
          else: # skip น้อยที่สุด ที่เป็นไปได้
              for i in range(p, len(text)):
                  ww = trie.prefixes(text[i:])
                  m = pat_eng.match(text[i:])
                  if ww or m:
                      break
              else:
                  i = len(text)
          w = text[p:i]
          words_at[p].append(w)
          yield w
          last_p = i
          heappush(q, i)

### heapq แทน set

In [ ]:
q = []   # min heap queue
for x in [4, 9, 2, 1, 5]:
  heappush(q, x)
q

[1, 2, 4, 9, 5]

In [ ]:
print("min of queue is", q[0])

min of queue is 1


In [ ]:
while q:
  print(heappop(q))

1
2
4
5
9


### Manual loop

In [ ]:
words_at = defaultdict(list)
q = [0]
last_p = 0

In [ ]:
# manual instead of while loop
print(q[0])
p = heappop(q)

for w in trie.prefixes(text[p:]):
  words_at[p].append(w)
  if p+len(w) not in q:
    heappush(q, p+len(w))
q

0


[1, 2, 6]

In [ ]:
# ส, สว
'สวัสดี' in wordlist

True

In [ ]:
print(q[0])
p = heappop(q)

for w in trie.prefixes(text[p:]):
  words_at[p].append(w)
  if p+len(w) not in q:
    heappush(q, p+len(w))
q

1


[2, 6]

In [ ]:
print(q[0])
p = heappop(q)

for w in trie.prefixes(text[p:]):
  words_at[p].append(w)
  if p+len(w) not in q:
    heappush(q, p+len(w))
q

2


[6]

In [ ]:
# if len(q) == 1:
#   q0 = q[0]
#   yield LatticeString(text[last_p:q0], serialize(last_p, q0))
#   last_p = q0
last_p, q[0]

(0, 6)

In [ ]:
words_at

defaultdict(list, {0: ['ส', 'สว', 'สวัสดี']})

### mm_path
ปรับจาก LatticeString ที่รวมทุกๆ path มาเป็น min แค่ path เดียว

In [ ]:
def serialize(p, p2):    # helper function
  for w in words_at[p]:
    p_ = p + len(w)
    if p_== p2:
      yield [w]
    elif p_ < p2:
      for path in serialize(p_, p2):
        yield [w]+path

In [ ]:
# maximal path ก็คือใช้ len เป็นตัวเลือก
min(serialize(0,6), key=len)

['สวัสดี']

ทดลองเสร็จแล้ว ก็ไปแก้ใน one cut ด้านบน

## ทดลอง

In [ ]:
list(onecut('ตามหามเหสี'))

['ตามหา', 'มเหสี']

## รวม TCC
หลักคือ คำนวณ tcc position ก่อน และสร้าง edge เฉพาะที่ลงพอดีตำแหน่งกัน

In [ ]:
pat_tcc = """\
เc็c
เcctาะ
เccีtยะ
เccีtย(?=[เ-ไก-ฮ]|$)
เccอะ
เcc็c
เcิc์c
เcิtc
เcีtยะ?
เcืtอะ?
เc[ิีุู]tย(?=[เ-ไก-ฮ]|$)
เctา?ะ?
cัtวะ
c[ัื]tc[ุิะ]?
c[ิุู]์
c[ะ-ู]t
c็
ct[ะาำ]?
แc็c
แcc์
แctะ
แcc็c
แccc์
โctะ
[เ-ไ]ct
""".replace('c','[ก-ฮ]').replace('t', '[่-๋]?').split()

def tcc(w):
    p = 0
    pat = re.compile("|".join(pat_tcc))
    while p<len(w):
        m = pat.match(w[p:])
        if m:
            n = m.span()[1]
        else:
            n = 1
        yield w[p:p+n]
        p += n

In [ ]:
list(tcc(text))

['ส',
 'วัส',
 'ดี',
 'ค',
 'รับ',
 ' ',
 'ส',
 'บา',
 'ย',
 'ดี',
 'ไห',
 'ม',
 'ค',
 'รับ']

In [ ]:
# ตำแหน่งที่อนุญาตให้ตัดได้
def tcc_pos(text):
  p_set = set()
  p = 0
  for w in tcc(text):
    p += len(w)
    p_set.add(p)
  return p_set

tcc_pos(text)

{1, 4, 6, 7, 10, 11, 12, 14, 15, 17, 19, 20, 21, 24}

In [ ]:
def mmcut(text):
  return list(onecut(text))

In [ ]:
# ตัวอย่างปัญหา ไม่ควร add 'จุ' เพราะไม่ตรง tcc_pos
mmcut('จุ๋ม')

['จุ', '๋ม']

In [ ]:
# แยก serialize ออกมา จะได้อ่าน code ง่ายขึ้น
def serialize(words_at, p, p2):
  # find path แบบ depth first
  for w in words_at[p]:
    p_ = p + len(w)
    if p_== p2:
      yield [w]
    elif p_ < p2:
      for path in serialize(words_at, p_, p2):
        yield [w]+path

In [ ]:
# ปรับ onecut ให้ใช้ tcc_pos
def onecut(text):
  words_at = defaultdict(list)  # main data structure
  allow_pos = tcc_pos(text)

  q = [0]       # min-heap queue
  last_p = 0    # last position for yield
  while q[0] < len(text):
      p = heappop(q)

      for w in trie.prefixes(text[p:]):
          p_ = p + len(w)
          if p_ in allow_pos:  # เลือกที่สอดคล้อง tcc
            words_at[p].append(w)
            if p_ not in q:
              heappush(q, p_)

      if len(q)==1:
          paths = serialize(words_at, last_p, q[0])
          for w in min(paths, key=len):
            yield w
          last_p = q[0]

      # กรณี len(q) == 0  คือ ไม่มีใน dict
      if len(q)==0:
          m = pat_eng.match(text[p:])
          if m: # อังกฤษ, เลข, ว่าง
              i = p + m.end()
          else: # skip น้อยที่สุด ที่เป็นไปได้
              for i in range(p, len(text)):
                  if i in allow_pos:   # ใช้ tcc ด้วย
                      ww = trie.prefixes(text[i:])
                      m = pat_eng.match(text[i:])
                      if ww or m:
                          break
              else:
                  i = len(text)
          w = text[p:i]
          words_at[p].append(w)
          yield w
          last_p = i
          heappush(q, i)

In [ ]:
mmcut(text)

['สวัสดี', 'ครับ', ' ', 'สบายดี', 'ไหม', 'ครับ']

In [ ]:
mmcut('จุ๋ม')

['จุ๋ม']

In [ ]:
mmcut('ไทยปน english ก็ได้นะ')

['ไทย', 'ปน', ' ', 'english', ' ', 'ก็ได้', 'นะ']

# สรุปรวม
copy code จากข้างบน เอาเฉพาะที่ใช้จริง

In [ ]:
import re
from collections import defaultdict
from heapq import heappush, heappop  # for priority queue
from marisa_trie import Trie

wordlist = [li.strip() for li in open('thaiword.txt')]
trie = Trie(wordlist)

# ช่วยตัดพวกภาษาอังกฤษ เป็นต้น
pat_eng = re.compile(r'''(?x)
[-a-zA-Z]+|   # english
\d[\d,\.]*|   # number
[ \t]+|       # space
\r?\n         # newline
''')

In [ ]:
# TCC
pat_tcc = """\
เc็c
เcctาะ
เccีtยะ
เccีtย(?=[เ-ไก-ฮ]|$)
เccอะ
เcc็c
เcิc์c
เcิtc
เcีtยะ?
เcืtอะ?
เc[ิีุู]tย(?=[เ-ไก-ฮ]|$)
เctา?ะ?
cัtวะ
c[ัื]tc[ุิะ]?
c[ิุู]์
c[ะ-ู]t
c็
ct[ะาำ]?
แc็c
แcc์
แctะ
แcc็c
แccc์
โctะ
[เ-ไ]ct
""".replace('c','[ก-ฮ]').replace('t', '[่-๋]?').split()

def tcc(w):
    p = 0
    pat = re.compile("|".join(pat_tcc))
    while p<len(w):
        m = pat.match(w[p:])
        if m:
            n = m.span()[1]
        else:
            n = 1
        yield w[p:p+n]
        p += n

def tcc_pos(text):
    p_set = set()
    p = 0
    for w in tcc(text):
        p += len(w)
        p_set.add(p)
    return p_set

In [ ]:
def serialize(words_at, p, p2):
  # find path ทั้งหมด แบบ depth first
  if p in words_at:
    for w in words_at[p]:
      p_ = p + len(w)
      if p_== p2:
        yield [w]
      elif p_ < p2:
        for path in serialize(words_at, p_, p2):
          yield [w]+path

In [ ]:
def onecut(text):
  words_at = defaultdict(list)  # main data structure
  allow_pos = tcc_pos(text)     # ตำแหน่งที่ตัด ต้องตรงกับ tcc

  q = [0]       # min-heap queue
  last_p = 0    # last position for yield
  while q[0] < len(text):
      p = heappop(q)

      for w in trie.prefixes(text[p:]):
          p_ = p + len(w)
          if p_ in allow_pos:  # เลือกที่สอดคล้อง tcc
            words_at[p].append(w)
            if p_ not in q:
              heappush(q, p_)

      # กรณี length 1 คือ ไม่กำกวมแล้ว ส่งผลลัพธ์ก่อนนี้คืนได้
      if len(q)==1:
          paths = serialize(words_at, last_p, q[0])
          for w in min(paths, key=len):
            yield w
          last_p = q[0]

      # กรณี length 0  คือ ไม่มีใน dict
      if len(q)==0:
          m = pat_eng.match(text[p:])
          if m: # อังกฤษ, เลข, ว่าง
              i = p + m.end()
          else: # skip น้อยที่สุด ที่เป็นไปได้
              for i in range(p+1, len(text)):
                  if i in allow_pos:   # ใช้ tcc ด้วย ทั้งจุดเริ่มและจบ
                      ww = [w for w in trie.prefixes(text[i:]) if (i+len(w) in allow_pos)]
                      m = pat_eng.match(text[i:])
                      if ww or m:
                          break
              else:
                  i = len(text)
          w = text[p:i]
          words_at[p].append(w)
          yield w
          last_p = i
          heappush(q, i)

# ช่วยให้ไม่ต้องพิมพ์ยาวๆ
def mmcut(text):
  return list(onecut(text))

In [ ]:
mmcut('สวัสดีครับ')   # ทำงานได้ถูกต้อง

['สวัสดี', 'ครับ']

In [ ]:
mmcut('จุ๋ม')

['จุ๋ม']